###  Юнит 3. Введение в машинное обучение 

#### Preproject

Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

 - **Restaurant_id** — идентификационный номер ресторана / сети ресторанов;
 - **City** — город, в котором находится ресторан;
 - **Cuisine Style** — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
 - **Ranking** — место, которое занимает данный ресторан среди всех ресторанов своего города;
 - **Rating** — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
 - **Price Range** — диапазон цен в ресторане;
 - **Number of Reviews** — количество отзывов о ресторане;
 - **Reviews** — данные о двух отзывах, которые отображаются на сайте ресторана;
 - **URL_TA** — URL страницы ресторана на TripAdvosor;
 - **ID_TA** — идентификатор ресторана в базе данных TripAdvisor.

Задачу, которая стоит перед вами, можно свести к трём пунктам:

 - Удалить из датафрейма столбцы, данные в которых представлены не числами (это вы уже сделали, и нужно просто повторить знакомые действия, но в этот раз выполнить данный шаг в последнюю очередь).
 - Избавиться от пропущенных (None) значений (на предыдущем шаге мы делали это самым грубым из всех возможных способов; сейчас попробуем подойти к процессу более гибко).
 - Создать новые столбцы с данными, используя для этого информацию, содержащуюся в других столбцах датафрейма (например, можно добавить столбец, сообщающий, сколько дней прошло со дня публикации последнего отзыва, отображённого на сайте).
 
С другой стороны, в этом задании масса подводных камней.

In [20]:
# импорт библиотек
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split # Загружаем специальный инструмент для разбивки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели 
from collections import Counter
import os
# import re
# import math
# import copy

In [22]:
df = pd.read_csv('main_task.csv')
display(df.head())

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [23]:
df.columns = df.columns.str.replace(' ','_')
df

,Restaurant_id,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


### ТРЕНИРОВКА

### 2.1 Задание 1
Какие столбцы НЕ содержат пропущенных (None) значений?

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine_Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price_Range        26114 non-null  object 
 6   Number_of_Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [25]:
df.columns[df.isna().sum() == 0]

Index(['Restaurant_id', 'City', 'Ranking', 'Rating', 'Reviews', 'URL_TA',
       'ID_TA'],
      dtype='object')

### 2.2 Задание 2
В каких столбцах данные хранятся в числовом формате?  
Ответ:  
- Ranking 
- Rating   
- Number of Reviews

In [26]:
df.columns[df.dtypes != np.object]

Index(['Ranking', 'Rating', 'Number_of_Reviews'], dtype='object')

### 2.3 Задание 3
В каких столбцах хранящиеся данные представляют собой список?

In [17]:
for vol in df.iloc[0]:
    if type(vol) == list: print(vol.index)


Ответ:  
   - Ни в одном

In [37]:
# сделаем из кода построения модели функцию, так как нам придется ее повторять 
def first_model(df):
    # Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
    X = df.drop(['Rating'], axis = 1)  
    y = df['Rating']  
    
    # Наборы данных с меткой "train" будут использоваться для обучения модели, 
    # "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

    # Создаём модель  
    regr = RandomForestRegressor(n_estimators=100)  
    
    # Обучаем модель на тестовом наборе данных  
    regr.fit(X_train, y_train)  
    
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
    y_pred = regr.predict(X_test) 

    # Сравниваем предсказанные значения (y_pred) с реальными (y_test), 
    # и смотрим насколько они в среднем отличаются  
    # Метрика называется Mean Absolute Error (MAE) и показывает 
    # среднее отклонение предсказанных значений от фактических.  
    print('MAE:', metrics.mean_absolute_error(y_test, y_pred)) 

In [27]:
# запускаем функцию с построением модели и оценкой МАЕ
first_model(df)

NameError: name 'first_model' is not defined

## 3.1 Вопрос для размышления  
По какой причине во время обучения модели возникла ошибка Value Error?  

***Ответ:*** - Ошибка возникла при обработке столбца со строковыми значениями. Следовательно, для обучения модели все данные должны быть переданы только в числовом формате. 


In [29]:
# заполняем пропуски 
# сначала заполним числовой ряд средними значениями 
df['Number_of_Reviews'] = df['Number_of_Reviews'].fillna(df['Number_of_Reviews'].mean())

In [30]:
# проверяем заполнение
df['Number_of_Reviews'].isnull().sum()

0

In [31]:
# теперь запольним Price Range
# сначала заменим все значения на числовые 
dic_value_Price = {'$':1,'$$ - $$$':2,'$$$$':3}
df['Price_Range']=df['Price_Range'].map(lambda x: dic_value_Price.get(x,x))
# потом заменим на самое популярное
df['Price_Range'] = df['Price_Range'].fillna(df["Price_Range"].value_counts().idxmax())

In [32]:
# проверяем заполнение
df['Price_Range'].isnull().sum()

0

In [33]:
# City переводим в числа с помощью метода get_dummies
df_city_dummies = pd.get_dummies(df['City'])
df = pd.concat([df,df_city_dummies], axis=1)

In [34]:
# на этом тренировочном этапе удалим все столбцы, 
# которые сложно заполнить или перевести в числа
df.drop(['Restaurant_id', 'City', 'Cuisine_Style', 'Reviews', 'URL_TA', 'ID_TA'], axis=1, inplace=True)

In [35]:
df

,Ranking,Rating,Price_Range,Number_of_Reviews,Amsterdam,Athens,Barcelona,Berlin,Bratislava,Brussels,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,5570.0,3.5,2.0,194.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1537.0,4.0,2.0,10.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,353.0,4.5,3.0,688.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3458.0,5.0,2.0,3.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,621.0,4.0,2.0,84.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500.0,4.5,2.0,79.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39996,6341.0,3.5,2.0,542.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39997,1652.0,4.5,2.0,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
39998,641.0,4.0,2.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
# повторяем построение модели
first_model(df)

MAE: 0.21757349999999998
